<h1>Crawling Data From Web PTA.Trunojoyo

Untuk melakukan proses crawling data pada Website, kita membutuhkan Library Scrapy. maka dari itu kita install dulu dengan cara ketik di CMD :

"pip install Scrapy"

Lalu kita import library tersebut dengan code dibawah ini:


In [ ]:
import scrapy

class Url(scrapy.Spider):
    name = "url"
    start_urls = []
    
    def __init__(self):
        url = 'https://pta.trunojoyo.ac.id/c_search/byprod/10/'
        for page in range(1,13):
            self.start_urls.append(url + str(page))
        
    def parse(self, response):
        for page in range(1,6):
            for url in response.css('#content_journal > ul'):
                yield {
                    'url' : url.css('li:nth-child('+str(page)+') > div:nth-child(3) > a ::attr(href)').extract()
                } 

Code diatas berfungsi untuk mengambil link tiap berita yang ingin kita ambil datanya.
Untuk menjalankan code diatas, kita harus save code tersebut dengan format .py agar dapat di run. setelah itu kita dapat menjalankan codenya dengan perintah "scrapy runspider -namafile.py -o -namafile.json".
untuk namafile.json adalah hasil run dari code diatas yang berisikan link dari berita yang akan kita ambil.

In [ ]:
import scrapy
import json

class Pta(scrapy.Spider):
    name = "pta"
    file_json = open("url.json")
    start_urls = json.loads(file_json.read())
    urls = []

    for i in range(len(start_urls)):
        b = start_urls[i]['url'][0]
        urls.append(b)
    
    def start_requests(self):
        for url in self.urls:
            yield scrapy.Request(url = url, callback = self.parse)
        
    def parse(self, response):
        # print(response.url)

        for jurnal in response.css('#content_journal > ul > li'):
            yield {
                'Judul':jurnal.css('div:nth-child(2) > a::text').get(),
                'Penulis':jurnal.css('div:nth-child(2) > div:nth-child(2) > span::text').get()[10:],
                'Dosbing_1':jurnal.css('div:nth-child(2) > div:nth-child(3) > span::text').get()[21:],
                'Dosbing_2':jurnal.css('div:nth-child(2) > div:nth-child(4) > span::text').get()[22:],
                'Abstrak_indo':jurnal.css('div:nth-child(4) > div:nth-child(2) > p::text').get(),
            }

Code diatas berfungsi untuk mengambil bagian yang perlu kita ambil pada tiap halaman PTA. sebelum itu kita harus import library Json terlebih dahulu, dikarenakan sebelumnya untuk link halaman PTA kita jadikan format .json

Pada code ini yang saya ambil adalah bagian Judul, Penulis, Dosen pembimbing 1, Dosen Pembimbing 2, dan Abstrak bahasa indonesianya dengan cara menginspect bagian yang akan kita ambil. setelah data di ambil, data tersebut kita jadikan format .csv untuk pemrosesan selanjutnya dengan cara run di terminal :

"scrapy runspider namafile.py -o namafile.csv"